In [1]:
#!/usr/bin/env python3

import torch
from torch import nn
import torch.nn.functional as F 

import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

import matplotlib.pyplot as plt
import numpy as np

import resource
rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (2048, rlimit[1]))

#~~~~Flags, options and hyperparameters~~~~

# Feature gate: Overtones Splits

import argparse


parser = argparse.ArgumentParser(prog='bsrnn')

parser.add_argument('--sample_rate', type=int, default=44100)
parser.add_argument('--generic_bands', type=str, default="N")
parser.add_argument('--chunk_size_in_seconds', type=int, default=1)
parser.add_argument('--n_fft', type=int, default=2048)
parser.add_argument('--feature_dim', type=int, default=128//4)
parser.add_argument('--num_blstm_layers', type=int, default=24//4)
parser.add_argument('--mlp_dim', type=int, default=512//2)
parser.add_argument('--batch_size', type=int, default=95)
parser.add_argument('--clip_grad_norm', type=int, default=5)
parser.add_argument('--max_epochs', type=int, default=500)
parser.add_argument('--portion', type=float, default=1.0)
parser.add_argument('--lr', type=float, default=0.001)

parser.add_argument('--musdbhq_location', type=str, default="/home/sahand/BandSplit-RNN/musdb18hq/")
parser.add_argument('--checkpoint_fp', type=str, default="./checkpoint_100_v7.pt")
parser.add_argument('--mode', type=str, default='transfer') # "train-base-model,transfer,infer"
parser.add_argument('--part', type=str, default="other")

args = parser.parse_args([])

fg_generic_bands = args.generic_bands == "Y"
sample_rate = args.sample_rate
chunk_size_in_seconds = args.chunk_size_in_seconds
chunk_size = chunk_size_in_seconds * sample_rate
n_fft = args.n_fft
win_length = n_fft
hop_length = win_length // 4
feature_dim = args.feature_dim
num_blstm_layers= args.num_blstm_layers
mlp_dim = args.mlp_dim
max_epochs = args.max_epochs
transfer_learning = args.mode == "transfer"
batch_size = args.batch_size
clip_grad_norm = args.clip_grad_norm
portion = args.portion
musdbhq_location = args.musdbhq_location
checkpoint_fp = args.checkpoint_fp
lr=args.lr
inference= args.mode == "infer"
train_base_model = args.mode == "train-base-model"
part = args.part

#~~~~Setup random generators~~~~
# TODO: from arg
seed = 123
torch.manual_seed(seed)
np.random.seed(seed)



#~~~~Pre-process and Post-process utils~~~~

def show_idp_audio(waveform):
    import IPython.display as idp
    n = 14
    return idp.display(idp.Audio(waveform[(3 * n) * sample_rate:(3 * (n + 1)) * sample_rate], rate=sample_rate))

def load_audio(path, visualize=False):
    waveform, sr = torchaudio.load(path)
    # Convert everthing to mono channel for simplicity
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0)
        # waveform is now a vector 
    # Resample everything to 44.1khz for simplicity
    resampler = T.Resample(sr, sample_rate, dtype=waveform.dtype)
    waveform = resampler(waveform)
    
    if visualize:
        # samplerate = 1/t
        # display the first 3 seconds
        show_idp_audio(waveform)
    
    return waveform

def rms_normalize(waveform, target_rms):
    current_rms = torch.sqrt(torch.mean(waveform**2))
    gain_factor = target_rms / (current_rms + 1e-10)
    normalized_waveform = waveform * gain_factor
    return normalized_waveform, gain_factor

def rms_denormalize(normalized_waveform, gain_factor):
    inverse_gain = 1 / gain_factor
    reversed_waveform = normalized_waveform * inverse_gain
    return reversed_waveform

def peak_normalize(waveform, target_peak):
    peak_value = torch.max(torch.abs(waveform))
    peak_gain_factor = target_peak / (peak_value + 1e-10)
    normalized_waveform = waveform * peak_gain_factor
    return normalized_waveform, peak_gain_factor

def peak_denormalize(normalized_waveform, peak_gain_factor):
    inverse_peak_gain = 1 / peak_gain_factor
    reversed_waveform = normalized_waveform * inverse_peak_gain
    return reversed_waveform

def inspect_waveform(waveform):
    transform = T.Loudness(sample_rate)
    return f"LKFS:{transform(waveform.unsqueeze(0))} max: {waveform.max()} min: {waveform.min()} avg: {waveform.mean()}"

def normalize_waveform(waveform, visualize=False):
    """ rms -> peak """
    # target rms can be anything. the important part here
    # is to be constant for all kind of songs

    if visualize:
        print("original: " + inspect_waveform(waveform))
        show_idp_audio(waveform)

    normalized_waveform, gain_factor = rms_normalize(waveform, target_rms=0.1)
    
    if visualize:
        print("rms_normalize: " + inspect_waveform(normalized_waveform))
        show_idp_audio(normalized_waveform)

    # setting target peak to 1.0 forces the values between -1.0 < y < 1.0
    normalized_waveform, peak_gain_factor = peak_normalize(normalized_waveform, target_peak=0.1)
    
    if visualize:
        print("peak_normalize: " + inspect_waveform(normalized_waveform))
        show_idp_audio(normalized_waveform)
    
    return normalized_waveform, gain_factor, peak_gain_factor

def de_normalize_waveform(waveform, gain_factor, peak_gain_factor, visualize=False):
    if visualize:
        print("de_normalize_waveform: " + inspect_waveform(waveform))
        show_idp_audio(waveform)
    
    waveform = peak_denormalize(waveform, peak_gain_factor)
    
    if visualize:
        print("peak_denormalize: " + inspect_waveform(waveform))
        show_idp_audio(waveform)
    waveform = rms_denormalize(waveform, gain_factor)
    
    if visualize:
        print("rms_denormalize: " + inspect_waveform(waveform))
        show_idp_audio(waveform)
    
    return waveform

def split(waveform, visualize=False, drop_last=False):
    # we have a vector by length n and we want to split it to even chunks by length of
    # chunk_size
    padding_length = (chunk_size - waveform.shape[0] % chunk_size) % chunk_size
    waveform = nn.functional.pad(waveform, (0, padding_length), 'constant', 0)
    # -1 means automatically infer based on other dims
    chunked_waveform = waveform.view(-1, chunk_size)
    
    if visualize:
        fig = plt.figure(constrained_layout=True, figsize=(16, 4))
        subfigs = fig.subfigures(2, 1).flat
        
        # first 3 chunk_size of waveform
        w = waveform[:3 * chunk_size].detach().numpy()
        ylim = [w.max() * 1.1, w.min() * 1.1]
        def time_axis(start, duration):
            return torch.arange(start * sample_rate, (duration + start) * sample_rate) / sample_rate
        axes = subfigs[0].subplots(1, 1)
        axes.plot(time_axis(0, 3), w, linewidth=0.3)
        axes.set_xlabel("time [s] for first 3 seconds")
        axes.set_ylim(ylim)
        
        # first 4 chunks + last chunk
        axes = subfigs[1].subplots(1, 5)
        for i, chunk in enumerate([0, 1, 3, 4, chunked_waveform.shape[0] - 1]): 
            axes[i].plot(time_axis(0, chunk_size_in_seconds), chunked_waveform[chunk], linewidth=0.3)
            axes[i].set_title(f"chunk {chunk}")
            axes[i].set_ylim(ylim)
    if drop_last:
        return chunked_waveform[:-1], 0
    return chunked_waveform, padding_length

def merge(chunks, padding_length):
    merged_waveform = torch.cat([torch.flatten(x) for x in chunks])
    return merged_waveform[:-padding_length]

def visualize_spectogram(chunk, chunk_stft, title='Spectogram'):
    import librosa
    fig, axis = plt.subplots(2, 1, figsize=(16, 5))
    noverlap = win_length - hop_length
    axis[0].imshow(librosa.power_to_db(chunk_stft.abs().detach().numpy() ** 2),\
                   origin="lower", aspect="auto", interpolation="nearest")
    axis[0].set_yscale("symlog")
    axis[0].set_title(title)
    if chunk is not None:
        axis[1].plot(chunk, linewidth=0.5)
        axis[1].grid(True)
        axis[1].set_xlim([0, len(chunk)])

def to_spectogram():
    transform_spectogram = T.Spectrogram(n_fft=n_fft, win_length=win_length, hop_length=hop_length,\
                                         window_fn=torch.hamming_window, power=None)
    def inner(chunk, visualize=False, title=''):
        chunk_stft = transform_spectogram(chunk)
        if visualize:
            visualize_spectogram(chunk, chunk_stft, title)

        return chunk_stft
    return inner

def from_spectogram():
    transform_inv_spectogram = T.InverseSpectrogram(n_fft=n_fft, win_length=win_length, hop_length=hop_length,\
                                                    window_fn=torch.hamming_window)

    def inner(chunk_stft, visualize=False):
        chunk = transform_inv_spectogram(chunk_stft)
    
        if visualize:
            visualize_spectogram(chunk.detach().numpy(), chunk_stft)

        return chunk
    return inner

class Vis:
    """ Generate model's visual graph """
    def visualize(self, input):
        from torchview import draw_graph
        y = self(input)
        x = draw_graph(self, input_data=input, device='meta', roll=True)
        print(f"--{input.shape}-->f(x)--{y.shape}-->")
        file = x.visual_graph.render(self._get_name())
        display(idp.FileLink("./" + file))

        
def get_microtone_name(semitones_from_A4, divisions_per_octave):
    notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    microtone_index = int((semitones_from_A4 + 9) * divisions_per_octave / 12) % divisions_per_octave
    octave = int((semitones_from_A4 + 9) // 12)
    note_index = microtone_index // (divisions_per_octave // 12)
    microtone_suffix = f"+{microtone_index % (divisions_per_octave // 12)}"
    return notes[note_index] + (microtone_suffix if microtone_suffix != "+0" else "") + str(octave)

def microtonal_notes(divisions_per_octave):
    A4_freq = 440.0
    min_freq = 20.0
    max_freq = 20000.0
    notes = []
    freqs = []
    current_freq = min_freq
    while current_freq <= max_freq:
        semitones_from_A4 = 12 * np.log2(current_freq / A4_freq)
        nearest_microtone = round(semitones_from_A4 * divisions_per_octave / 12)
        nearest_freq = A4_freq * (2 ** (nearest_microtone / divisions_per_octave))

        # Check if the frequency is a microtonal (not a standard semitone)
        if nearest_microtone % (divisions_per_octave // 12) != 0:
            note_name = get_microtone_name(nearest_microtone / (divisions_per_octave / 12), divisions_per_octave)
            notes.append(note_name)
            freqs.append(nearest_freq)

        # Move to the next microtone
        current_freq = A4_freq * (2 ** ((nearest_microtone + 1) / divisions_per_octave))
    return (notes, freqs)

def plot_custom_labeled_attention_distribution(splits):
    start_freq = 0
    events = []
    for end_freq, step_size in splits:
        while start_freq < end_freq:
            start_freq += step_size
            events.append(start_freq)

    notes, positions = microtonal_notes(24)
    print(len(events))

    # Plotting
    plt.figure(figsize=(18, 6))
    plt.xscale("log")
    plt.yticks([])
    plt.xticks(positions, notes, rotation=90, fontsize=9)
    plt.eventplot(events, orientation='horizontal', colors='b')
    

    plt.show()
    
def evenly_skip_elements(input_list, k):
    step = len(input_list) / k
    new_list = [input_list[int(i * step)] for i in range(k)]

    return new_list

def create_evenly_distributed_splits(num_splits):
    _, freqs = microtonal_notes(24)
    splits = []
    last_freq = 0
    for freq in evenly_skip_elements(freqs, num_splits):
        splits.append((freq, freq - last_freq))
        last_freq = freq
    return splits

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#~~~~Model definition~~~~

# Numbers are exctracted from the paper
splits_generic = create_evenly_distributed_splits(41)
splits_v7 = [
   # below 1kh, bandwidth 100hz
   (1000, 100),
   # above 1kh and below 4khz, bandwidth 250hz
   (4000, 250),
   (8000, 500),
   (16000, 1000),
   (20000, 2000),
]

#~~~~Module 1

temporal_dim = int(np.ceil(chunk_size / T.Spectrogram(n_fft=n_fft, win_length=win_length, hop_length=hop_length).hop_length))

class BandSplit(nn.Module, Vis):
    
    def __init__(self, splits=splits_generic if fg_generic_bands else splits_v7, fully_connected_out=feature_dim):
        super(BandSplit, self).__init__()

        #### Make splits
        # convert fft to freq
        freqs = sample_rate * torch.fft.fftfreq(n_fft)[:n_fft // 2 + 1]
        freqs[-1] = sample_rate // 2
        indices = []
        start_freq, start_index = 0, 0
        for end_freq, step in splits:
            bands = torch.arange(start_freq + step, end_freq + step, step)
            start_freq = end_freq
            for band in bands:
                end_index = freqs[freqs < band].shape[0]
                if end_index != start_index or not fg_generic_bands:
                    indices.append((start_index, end_index))
                start_index = end_index
        indices.append((start_index, freqs.shape[0]))
        self.band_indices = indices
        self.fully_connected_out = fully_connected_out
        
        self.layer_norms = nn.ModuleList([
            # * 2 is for added dim of view_as_real
            nn.LayerNorm([(band_end - band_start) * 2, temporal_dim])
            for band_start, band_end in self.band_indices
        ])
        
        self.layer_fcs =  nn.ModuleList([
            # * 2 is for added dim of view_as_real
            nn.Linear((band_end - band_start) * 2, self.fully_connected_out)
            for band_start, band_end in self.band_indices
        ])

    def forward(self, chunk_ftt):
        batch_size = chunk_ftt.size(0)
        stack = []
        # TODO: can i vectorize this loop?
        for i, (band_start, band_end) in enumerate(self.band_indices):
            band = chunk_ftt[:, band_start:band_end, :]
            # band is shape of (B, F, T)
            band = torch.view_as_real(band) # (B, F, T, 2)
            # convert to (B, 2, F, T) to be able to feed it to the norm
            band = band.permute(0, 3, 1, 2)
            
            # norm is (..., F, T) and fc is (Fxfully_connected_out)
            # we should make norm (..., T, F) in order to feed it to the fc
            band = band.reshape(batch_size, -1, band.size(-1)) # -1 = T
            norm = self.layer_norms[i](band)
            
            norm = norm.transpose(-1, -2).contiguous()
            fc_y = self.layer_fcs[i](norm)
            
            stack.append(fc_y)
        return torch.stack(stack, dim=1)

#~~~~Module 2

class RNN(nn.Module):
    def __init__(self, input_dim_size):
        super(RNN, self).__init__()
        self.input_dim_size = input_dim_size
        # paper specified group norm
        self.norm = nn.ModuleList([nn.GroupNorm(self.input_dim_size, self.input_dim_size) for _ in range(2)])
        self.blstm = nn.ModuleList([nn.LSTM(self.input_dim_size, self.input_dim_size,\
                                            bidirectional=True, batch_first=True) for _ in range(2)])
        self.fc = nn.ModuleList([nn.Linear(self.input_dim_size * 2, self.input_dim_size) for _ in range(2)])
    
    def forward(self, x):
        # input is b, bands(K), temporal_dim(t), input_dim_size
        
        
        # First loops converts the shape to [B, T, K, N]
        # and the second loop converts it back to [B, K, T, N]
        for i in range(2):
            B, K, T, N = x.shape
            out = x.view(B * K, T, N)
            out = self.norm[i](out.transpose(-1, -2)).transpose(-1, -2)
            out = self.blstm[i](out)[0]
            out = self.fc[i](out)
            x = out.view(B, K, T, N) + x
            x = x.permute(0, 2, 1, 3).contiguous()
        
        return x



class BandSequence(nn.Module, Vis):
    
    def __init__(self, input_dim_size, num_layers=num_blstm_layers):
        super(BandSequence, self).__init__()
        self.rnns = nn.Sequential(*[RNN(input_dim_size=input_dim_size) for _ in range(num_layers)])
        
    def forward(self, x):
        # (bands, temporal_dim, fc_out)
        return self.rnns(x)

#~~~~Module 3    

class MaskEstimation(nn.Module, Vis):
    def __init__(self, band_indices, fully_connected_out):
        super(MaskEstimation, self).__init__()
        
        max_indice_diff = max([e - s for s, e in band_indices])
        num_hiddens = lambda e, s: 3 * (max_indice_diff - (e - s) + 1)
        self.layers = nn.ModuleList([
            nn.Sequential(
                nn.LayerNorm([temporal_dim, fully_connected_out]),
                nn.Linear(fully_connected_out, mlp_dim),
                nn.Tanh(),
                # double the output dim to use in GLU
                # the extra *2 is for returning as complex
                nn.Linear(mlp_dim, (e - s) * 2 * 2),
                nn.GLU()
            )
            for s, e in band_indices
        ])
        
    def forward(self, x):
        # (b, k, temporal_dim, fc_out)
        parts = []
        for i in range(x.shape[1]):
            y = self.layers[i](x[:, i]).contiguous()
            B, T, F = y.shape
            y = y.permute(0, 2, 1).contiguous() # B F T
            # basically halve the freq dim and use it for phasee
            y = y.view(B, 2, F // 2, T) # (B, 2, F, T)
            y = y.permute(0, 2, 3, 1) # (B, F, T, 2)
            y = torch.view_as_complex(y.contiguous())
            
            parts.append(y)
        
        # (b, f, t)
        return torch.cat(parts, dim=-2)
    

#~~~~Module BSRNN

# Putting everything together!

class BSRNN(nn.Module, Vis):
    
    def __init__(self, num_sources=1):
        super(BSRNN, self).__init__()
        
        self.split = BandSplit()
        self.sequence = BandSequence(input_dim_size=self.split.fully_connected_out)
        
        self.masks = nn.ModuleList([MaskEstimation(band_indices=self.split.band_indices, 
                                                    fully_connected_out=self.split.fully_connected_out) 
                                    for _ in range(num_sources)])

    def forward(self, chunk_fft):
        
        mean = chunk_fft.mean(dim=(1, 2), keepdim=True)
        std = chunk_fft.std(dim=(1, 2), keepdim=True)
        chunk_fft = (chunk_fft - mean) / (std + 1e-5)
        
        y = self.split(chunk_fft)
        y = self.sequence(y)
        masks = torch.stack([mask(y) for mask in self.masks], dim=0)
        
        masks = (masks * std) + mean

        return masks

class MSSBandSplitRNN(nn.Module, Vis):
    def __init__(self):
        super(MSSBandSplitRNN, self).__init__()
        self.to_spectogram = to_spectogram()
        self.from_spectogram = from_spectogram()
        self.bsrnn = BSRNN()
        
    def forward(self, waveform):
        """ Waveform in -> Waveform out :) """
        
        # 1) normalize
        # 2) split
        # 3) feed to bsrnn
        # 4) convert spectogram to audio
        # 5) merge all splits
        # 6) de-normalize
        
        normal_waveform, gain_factor, peak_gain_factor = normalize_waveform(waveform)
        splits, padding_length = split(normal_waveform)
        masked_splits = [[] for _ in range(len(splits))]
        for i, x_split in enumerate(splits):
            split_stft = self.to_spectogram(x_split)
            masks = self.bsrnn(split_stft.unsqueeze(0))[0]
            
            for source in masks:
                wave = self.from_spectogram(masked_complex)
                masked_splits[i].append(wave)
        
        sources = []
        for masked_splits_in_source in zip(*masked_splits):
            sources.append(masked_splits_in_source)
        
        masked_waveforms = [merge(x, padding_length) for x in sources]
        y = [de_normalize_waveform(x, gain_factor, peak_gain_factor) in masked_waveforms]
        return y
    

#~~~~Training

#~~Loss,Eval

class CustomLoss(nn.Module):
    
    def __init__(self):
        super(CustomLoss, self).__init__()
        self.mae_stft_real = nn.L1Loss()
        self.mae_stft_imag = nn.L1Loss()
        self.mae_inv_stft  = nn.L1Loss()
    
    def forward(self, pred_stft, target_stft,  pred_inv_stft, target_inv_stft):
        loss_r = self.mae_stft_real(pred_stft.real, target_stft.real)
        loss_i = self.mae_stft_imag(pred_stft.imag, target_stft.imag)
        loss_t = self.mae_inv_stft(pred_inv_stft, target_inv_stft)
        loss = loss_r + loss_i + loss_t
        return loss

def compute_usdr(pred, target, delta = 1e-7):
    if pred.shape[0] < target.shape[0]:
        padding = target.shape[0] - pred.shape[0]
        pred = torch.nn.functional.pad(pred, (0, padding), "constant", 0)
    num = torch.sum(torch.square(target))
    den = torch.sum(torch.square(target - pred))
    num += delta
    den += delta
    usdr = 10 * torch.log10(num / den)
    return usdr.mean()

#~~Dataloader
import os
import random
import math

all_parts = ["bass", "drums", "other", "vocals"]

class Dataset(torch.utils.data.IterableDataset):
    def __init__(self, parts=all_parts, dir_=musdbhq_location, validation=False, portion=1):
        super(Dataset, self).__init__()
        path = dir_ + ("test" if validation else "train")
        self.parts = parts
        self.files = [entry for entry in os.scandir(path) if entry.is_dir()]
        random.shuffle(self.files) # we are seeding the random gen manually so this is fine.
        self.files = self.files[:int(len(self.files) * portion)]
        self.loaded = {}
        torch.set_default_device('cpu')
        self.to_stft = to_spectogram()
        print("pre-loading")
        targets = [f"{d.path}/{part}.wav" for part in self.parts for d in self.files]
        [targets.append(f"{d.path}/mixture.wav") for d in self.files]
        from tqdm import tqdm
        self.cache = True
        for x in tqdm(targets[:int(len(targets) * 0.2)]):
            self.load_audio(x)
        self.cache = False
        print("pre-loading done")

    def load_audio(self, path):
        if path in self.loaded:
            return self.loaded[path]
        x = load_audio(path)
        x, _, _ = normalize_waveform(x)
        x, _ = split(x, drop_last=True)
        x = torch.stack([self.to_stft(k) for k in x], dim=0)
        if self.cache:
            self.loaded[path] = x
        return x
    
    def iterator(self):
        torch.set_default_device('cpu')
        
        for i in range(self.start_index, self.end_index):
            d = self.files[i]
            
            mixture = f"{d.path}/mixture.wav"
            targets = [f"{d.path}/{part}.wav" for part in self.parts]
            
            mix_stft_splits = self.load_audio(mixture)
            target_stfts_splits = [self.load_audio(target) for target in targets]
            
            for i, mix_stft in enumerate(mix_stft_splits):
                target_stfts = [x[i] for x in target_stfts_splits]
                out = (d.path, mix_stft, target_stfts)
                yield out

    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is not None:
            total_files = len(self.files)
            per_worker = int(math.ceil(total_files / float(worker_info.num_workers)))
            self.start_index = worker_info.id * per_worker
            self.end_index = min(self.start_index + per_worker, total_files)
        else:
            self.start_index = 0
            self.end_index = len(self.files)

        return iter(self.iterator())

    

In [2]:


num_workers=1
parts = all_parts if train_base_model else [part]
t_dataset = Dataset(validation=False, parts=parts, portion=portion)

pre-loading


100%|███████████████████████████████████████████████████████████████████████| 40/40 [00:08<00:00,  4.68it/s]

pre-loading done


In [3]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [4]:
v_dataset = Dataset(validation=True, parts=parts, portion=portion)

pre-loading


100%|███████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.62it/s]

pre-loading done


In [5]:
import gc
torch.cuda.empty_cache()
gc.collect()
# del val_loader
print(parts)

['other']


In [6]:
import gc
torch.cuda.empty_cache()
gc.collect()
# del trainer
from tqdm.auto import tqdm
import time

def collate_fn(batch):
    # (d.path, mix_stft (1025, 87), target_stfts (4, 1025, 87))
    b_mix_stft = []
    b_target_stfts = [[] for _ in range(len(batch[0][2]))]
    
    for x, mix, target in batch:
        b_mix_stft.append(mix)
        for i, t in enumerate(target):
            b_target_stfts[i].append(t)
    # (B, 1025, 87), (4, B, 1025, 87)
    return torch.stack(b_mix_stft), [torch.stack(x) for x in b_target_stfts]

batch_size = 270
train_loader = torch.utils.data.DataLoader(t_dataset, num_workers=num_workers, 
                                          batch_size=batch_size, drop_last=True,
                                          prefetch_factor=4,
                                           collate_fn=collate_fn,
                                          persistent_workers=True)
val_loader   = torch.utils.data.DataLoader(v_dataset, num_workers=num_workers, 
                                          batch_size=batch_size, drop_last=True,
                                          prefetch_factor=4,
                                           collate_fn=collate_fn,
                                          persistent_workers=True)


# because we will load from base model in transfer learning
model = BSRNN(num_sources=len(all_parts) if transfer_learning else len(parts)).to('cuda')
import datetime

fg_prefix = "generic-split" if fg_generic_bands else "v7-split"
name_prefix = "base-model" if train_base_model else f"base-model-to-{part}"

name = f"{fg_prefix}-{name_prefix}-portion-{portion}"
prefix=f"./train-logs/{name}-{datetime.datetime.now()}"
if not os.path.exists(name):
    os.makedirs(prefix)
print(f"run name={prefix}")


#~~~~Train

from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage
from ignite.handlers import ModelCheckpoint, Checkpoint, DiskSaver, global_step_from_engine
from ignite.contrib.handlers import TensorboardLogger, global_step_from_engine, TensorboardLogger
from ignite.contrib.handlers import ProgressBar
from ignite.handlers import Timer
#~~Ignite setup

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = CustomLoss()

torch.set_default_device('cuda')
inv_stft_gpu = from_spectogram()


def train_step(engine, batch):
    # start = time.time()
    model.train()
    # print(f"model.train(){time.time() - start}s");start = time.time()
    optimizer.zero_grad()
    # print(f"zero grad{time.time() - start}s");start = time.time()
    mix_stft, mask_stfts = batch
    mix_stft, mask_stfts = mix_stft.to('cuda'), mask_stfts
    mask_stfts = [x.to('cuda') for x in mask_stfts]
    # print(f"batch{time.time() - start}s");start = time.time()
    
    y_masks = model(mix_stft) # (num_sources, B, 1025, 87)
    # print(f"ff{time.time() - start}s");start = time.time()
    inv_y_masks = [inv_stft_gpu(y_mask) for y_mask in y_masks]
    # print(f"inv{time.time() - start}s");start = time.time()
    # (num_sources, B, 1025, 87)
    inv_mask_stfts = [inv_stft_gpu(mask_stft) for mask_stft in mask_stfts]
    # print(f"inv masks{time.time() - start}s");start = time.time()
    
    total_loss = 0 # total loss for all heads
    for y_mask, mask_stft, inv_y_mask, inv_mask_stft in zip(y_masks, mask_stfts, inv_y_masks, inv_mask_stfts):
        total_loss += loss_fn(y_mask, mask_stft, inv_y_mask, inv_mask_stft)
    # print(f"loss{time.time() - start}s");start = time.time()
    total_loss.backward()
    # print(f"backward{time.time() - start}s");start = time.time()
    
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
    optimizer.step()
    # print(f"opt{time.time() - start}s");start = time.time()
    
    usdr_values = [compute_usdr(inv_y_mask, inv_mask_stft).item() for \
                      inv_y_mask, inv_mask_stft in zip(inv_y_masks, inv_mask_stfts)]
    # print(f"score{time.time() - start}s");start = time.time()
    return {'loss': total_loss.item(), 'usdr': np.mean(usdr_values)}


trainer = Engine(train_step)

running_avg_usdr = RunningAverage(output_transform=lambda x: x['usdr'])
running_avg_usdr.attach(trainer, 'running_avg_usdr')
running_avg_loss = RunningAverage(output_transform=lambda x: x['loss'])
running_avg_loss.attach(trainer, 'running_avg_loss')

tb_logger = TensorboardLogger(log_dir=f"{prefix}/tb")
tb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED,
    tag="training",
    output_transform=lambda x: {"running_avg_loss": trainer.state.metrics['running_avg_loss'], 
                                "running_avg_usdr": trainer.state.metrics['running_avg_usdr']}
)

def eval_step(engine, batch):
    model.eval()
    with torch.no_grad():
        mix_stft, mask_stfts = batch
        mix_stft, mask_stfts = mix_stft.to('cuda'), mask_stfts
        mask_stfts = [mask_stft.to('cuda') for mask_stft in mask_stfts]

        y_masks = model(mix_stft)
        inv_y_masks = [inv_stft_gpu(y_mask) for y_mask in y_masks]
        inv_mask_stfts = [inv_stft_gpu(mask_stft) for mask_stft in mask_stfts]
        
        
        usdr_values = [compute_usdr(inv_y_mask, inv_mask_stft).item() for \
                      inv_y_mask, inv_mask_stft in zip(inv_y_masks, inv_mask_stfts)]
        
        return {"usdr": np.mean(usdr_values)}

evaluator = Engine(eval_step)

running_avg_usdr_eval = RunningAverage(output_transform=lambda x: x['usdr'])
running_avg_usdr_eval.attach(evaluator, 'running_avg_usdr_eval')

iter_timer = Timer(average=True)
epoch_timer = Timer(average=False)

@evaluator.on(Events.COMPLETED)
def log_results(engine):
    usdr = engine.state.metrics['running_avg_usdr_eval']
    print(f"Test Results - Avg usdr: {usdr} - iter_timer:{iter_timer.value()}s - epoch_timer:{epoch_timer.value()}")

bar = ProgressBar(persist=False)
bar.attach(evaluator, metric_names=['running_avg_usdr_eval'])

@trainer.on(Events.EPOCH_COMPLETED)
def train_epoch_completed(engine):
    epoch = engine.state.epoch
    start = time.time()
    evaluator.run(val_loader)
    tb_logger.add_scalar("evaluation/running_avg_usdr_eval", evaluator.state.metrics['running_avg_usdr_eval'], epoch)


iter_timer.attach(
    trainer,
    start=Events.STARTED,
    resume=Events.ITERATION_STARTED,
    pause=Events.ITERATION_COMPLETED,
    step=Events.ITERATION_COMPLETED
)

epoch_timer.attach(
    trainer,
    start=Events.EPOCH_STARTED,
    step=Events.EPOCH_COMPLETED,
)

# Model Checkpointing
to_save = {
    'trainer': trainer,
    'model': model,
    'optimizer': optimizer,
    'running_avg_usdr': running_avg_usdr,
    'running_avg_loss': running_avg_loss
}
checkpoint_handler = Checkpoint(
    to_save,
    DiskSaver(f'{prefix}/models', create_dir=True),
    n_saved=25,
    global_step_transform=global_step_from_engine(trainer)
)

trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler)

# Load checkpoint
if os.path.isfile(checkpoint_fp):
    checkpoint = torch.load(checkpoint_fp)
    Checkpoint.load_objects(to_load=to_save, checkpoint=checkpoint)


# Progress Bar

if transfer_learning:
    new_model = BSRNN(num_sources=1).to('cuda')
    print(f"Params checkpoint -> {count_parameters(model)}")
    new_model.split = model.split
    new_model.sequence = model.sequence
    new_model.mask = nn.ModuleList([MaskEstimation(
     band_indices=new_model.split.band_indices, 
     fully_connected_out=new_model.split.fully_connected_out   
    )])
    model = new_model
    print(f"Params before freeze -> {count_parameters(model)}")
    # Freeze first two modules. keep the mask estimation
    for param in model.split.parameters():
        param.requires_grad = False
    for param in model.sequence.parameters():
        param.requires_grad = False
    for param in model.masks[0].parameters():
        param.requires_grad = True
    running_avg_usdr.reset()
    running_avg_loss.reset()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    trainer.state.max_epochs = None
    print(f"Params after freeze -> {count_parameters(model)}")


pbar = ProgressBar(persist=True)
pbar.attach(trainer, metric_names=['running_avg_loss', 'running_avg_usdr'])
# Run Training
print(f"Parameter count {count_parameters(model)}")
trainer.run(train_loader, max_epochs=max_epochs)
print(prefix)    
print("Done!")

run name=./train-logs/v7-split-base-model-to-other-portion-1.0-2023-12-31 09:19:13.619384


2023-12-31 09:19:14.856442: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-31 09:19:14.856488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-31 09:19:14.904089: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-31 09:19:14.987487: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-31 09:19:15.740573: W tensorflow/compiler/tf2

Params checkpoint -> 7165516
Params before freeze -> 3908804
Params after freeze -> 3256712
Parameter count 3256712


[1/?]   0%|           [00:00<?]

[1/?]   0%|           [00:00<?]

...

[1/?]   0%|           [00:00<?]

Test Results - Avg usdr: 1.3551197350583422 - iter_timer:0.47855622577480295s - epoch_timer:1662.564728409001


[1/84]   1%|1          [00:00<?]

Engine run is terminating due to exception: 


KeyboardInterrupt: 

In [ ]:
# 17552144
#  7165516
#  6997228